In [1]:
# importando pacotes
from ucimlrepo import fetch_ucirepo 
import pandas
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

In [2]:
# CARREGANDO DADOS

# importando dataset
dataset = fetch_ucirepo(id=320) # 649 registros e 30 atributos
  
# coletando as informações
data_frame = dataset.data.original

In [3]:
# TRATANDO DADOS

print("PRÉ TRATAMENTO: ", len(data_frame))

# removendo colunas com muitos nulos
tolerancia = len(data_frame) * 0.7
data_frame = data_frame.dropna(axis=1, thresh=tolerancia)

# removendo duplicados
print("VALORES DUPLICADOS: ", data_frame.duplicated().sum())
data_frame = data_frame.drop_duplicates()

# convertendo colunas categóricas em valores inteiros
conversores = {}
for coluna in data_frame.columns:
    if (data_frame[coluna].dtype == type(object)):
        conversor = LabelEncoder()
        data_frame[coluna] = conversor.fit_transform(data_frame[coluna])
        conversores[coluna] = conversor

print("PÓS TRATAMENTO: ", len(data_frame))

PRÉ TRATAMENTO:  649
VALORES DUPLICADOS:  0
PÓS TRATAMENTO:  649


In [4]:
print(data_frame.columns)

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object')


In [5]:
# DIVIDINDO O DATASET TRATADO

atributos = data_frame.drop(["G1", "G2", "G3"], axis=1)
respostas = data_frame[["G1", "G2", "G3"]]

a_treino, a_teste, r_treino, r_teste = train_test_split(atributos, respostas, test_size=0.3, random_state=42)


In [6]:
# CRIANDO PARÂMETROS PARA OS MODELOS

lista_parametros = [
    {"id": "1", "n_estimators": 10, "criterion": "gini", "max_depth": None, "min_samples_split": 2, "min_samples_leaf": 5},
    {"id": "2", "n_estimators": 25, "criterion": "entropy", "max_depth": 5, "min_samples_split": 10, "min_samples_leaf": 5},
    {"id": "3", "n_estimators": 75, "criterion": "gini", "max_depth": None, "min_samples_split": 2, "min_samples_leaf": 5},
    {"id": "4", "n_estimators": 100, "criterion": "entropy", "max_depth": 5, "min_samples_split": 10, "min_samples_leaf": 5},
]

In [7]:
# APLICANDO MODELO

# criando a lista de resultados finais
lista_resultados = []

for parametros in lista_parametros:
    # criando classificador
    classificador = RandomForestClassifier(
        n_estimators=parametros["n_estimators"], 
        criterion=parametros["criterion"], 
        max_depth=parametros["max_depth"], 
        min_samples_split=parametros["min_samples_split"], 
        min_samples_leaf=parametros["min_samples_leaf"], 
        random_state=42
    )

    # treinando o modelo
    classificador.fit(a_treino, r_treino)

    # prevendo respostas
    r_previsao = classificador.predict(a_teste)

    # calculando métricas
    acuracia_G1 = accuracy_score(r_teste.iloc[:, 0], r_previsao[:, 0])
    acuracia_G2 = accuracy_score(r_teste.iloc[:, 1], r_previsao[:, 1])
    acuracia_G3 = accuracy_score(r_teste.iloc[:, 2], r_previsao[:, 2])
    f1_G1 = f1_score(r_teste.iloc[:, 0], r_previsao[:, 0], average="weighted")
    f1_G2 = f1_score(r_teste.iloc[:, 1], r_previsao[:, 1], average="weighted")
    f1_G3 = f1_score(r_teste.iloc[:, 2], r_previsao[:, 2], average="weighted")

    # salvando resultados
    lista_resultados.append([parametros["id"], acuracia_G1, f1_G1, acuracia_G2, f1_G2, acuracia_G3, f1_G3])

In [8]:
# mostrando resultados
tabela_resultados = pandas.DataFrame(lista_resultados, columns=["Identificador", "AcuráciaG1", "F1-scoreG1", "AcuráciaG2", "F1-scoreG2", "AcuráciaG3", "F1-scoreG3"])
print(tabela_resultados)

  Identificador  AcuráciaG1  F1-scoreG1  AcuráciaG2  F1-scoreG2  AcuráciaG3  \
0             1    0.174359    0.153729    0.143590    0.134743    0.174359   
1             2    0.143590    0.117841    0.210256    0.175248    0.169231   
2             3    0.143590    0.120410    0.205128    0.168468    0.169231   
3             4    0.143590    0.114371    0.189744    0.158120    0.174359   

   F1-scoreG3  
0    0.142013  
1    0.109988  
2    0.122499  
3    0.109010  
